In [1]:
!nvidia-smi

Sun Jan 18 14:24:43 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


# Задание 1

In [3]:
%%writefile assignment3_task1.cu
#include <cuda_runtime.h> // Основной заголовочный файл CUDA для работы с рантаймом
#include <iostream>       // Стандартный ввод-вывод C++
#include <random>         // Библиотека для генерации случайных чисел
#include <cstdlib>        // Стандартная библиотека (для malloc, exit)

// Константы программы
#define N 1000000         // Размер массива (1 миллион элементов)
#define BLOCK_SIZE 256    // Количество потоков в одном блоке CUDA
#define RAND_MIN_VAL 1    // Минимальное значение для рандома
#define RAND_MAX_VAL 100000 // Максимальное значение для рандома

using namespace std;

// ===============================
// Макрос для проверки ошибок CUDA
// ===============================
#define CUDA_CHECK(call) \
do { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        cerr << "CUDA error: " << cudaGetErrorString(err) \
             << " at " << __FILE__ << ":" << __LINE__ << endl; \
        exit(EXIT_FAILURE); \
    } \
} while (0)

// ===============================
// Ядра (Kernels)
// ===============================

// Ядро, использующее только глобальную память GPU
__global__ void multiply_global(float* data, float value, int n) {
    // Вычисляем уникальный индекс потока во всей сетке
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверка границ: работаем только если индекс меньше размера массива
    if (idx < n) {
        data[idx] *= value; // Прямое чтение и запись в глобальную память
    }
}

// Ядро, использующее разделяемую (shared) память
__global__ void multiply_shared(float* data, float value, int n) {
    // Объявляем массив в разделяемой памяти (быстрая память внутри блока)
    __shared__ float buf[BLOCK_SIZE];

    // Глобальный индекс элемента
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < n) {
        // 1. Копируем данные из медленной глобальной памяти в быструю разделяемую
        buf[threadIdx.x] = data[idx];

        // Синхронизация: ждем, пока все потоки блока заполнят буфер
        __syncthreads();

        // 2. Выполняем операцию в разделяемой памяти
        buf[threadIdx.x] *= value;

        // Синхронизация: ждем завершения вычислений перед записью обратно
        __syncthreads();

        // 3. Копируем результат обратно в глобальную память
        data[idx] = buf[threadIdx.x];
    }
}

// ===============================
// Вспомогательная функция для печати краев массива
// ===============================
void print_edges(const float* a, const string& msg) {
    cout << msg << "\nFirst 5: ";
    for (int i = 0; i < 5; i++) cout << a[i] << " "; // Печать первых 5 элементов
    cout << "\nLast 5:  ";
    for (int i = N - 5; i < N; i++) cout << a[i] << " "; // Печать последних 5
    cout << "\n\n";
}

// ===============================
// Основная функция
// ===============================
int main() {
    // Проверка наличия доступных GPU
    int deviceCount = 0;
    CUDA_CHECK(cudaGetDeviceCount(&deviceCount));
    if (deviceCount == 0) {
        cerr << "ERROR: No CUDA GPU found. Enable GPU in Colab.\n";
        return 1;
    }

    float *h_data, *h_res, *d_data; // Указатели для хоста (CPU) и устройства (GPU)
    float multiplier = 2.5f;        // Число, на которое будем умножать
    size_t size = N * sizeof(float); // Общий размер выделяемой памяти в байтах

    // Инициализация генератора случайных чисел
    random_device rd;
    mt19937 gen(rd());
    uniform_int_distribution<> dist(RAND_MIN_VAL, RAND_MAX_VAL);

    // Выделение памяти на хосте (RAM)
    h_data = (float*)malloc(size);
    h_res  = (float*)malloc(size);

    // Заполнение исходного массива случайными числами
    for (int i = 0; i < N; i++)
        h_data[i] = static_cast<float>(dist(gen));

    // Выделение памяти на устройстве (VRAM видеокарты)
    CUDA_CHECK(cudaMalloc(&d_data, size));

    // Расчет параметров сетки: сколько блоков нужно для N элементов
    int grid = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // Создание событий CUDA для замера времени выполнения
    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));

    float time_global = 0.0f;
    float time_shared = 0.0f;

    // ===============================
    // ТЕСТ 1: ГЛОБАЛЬНАЯ ПАМЯТЬ
    // ===============================
    cout << "=== GLOBAL MEMORY VERSION ===\n";
    print_edges(h_data, "Before");

    // Копируем данные с CPU на GPU
    CUDA_CHECK(cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice));

    CUDA_CHECK(cudaEventRecord(start)); // Старт таймера
    multiply_global<<<grid, BLOCK_SIZE>>>(d_data, multiplier, N); // Запуск ядра
    CUDA_CHECK(cudaEventRecord(stop));  // Стоп таймера

    CUDA_CHECK(cudaGetLastError());     // Проверка на ошибки при запуске ядра
    CUDA_CHECK(cudaEventSynchronize(stop)); // Ждем завершения работы GPU
    CUDA_CHECK(cudaEventElapsedTime(&time_global, start, stop)); // Считаем время

    // Копируем результат обратно на CPU для проверки
    CUDA_CHECK(cudaMemcpy(h_res, d_data, size, cudaMemcpyDeviceToHost));
    print_edges(h_res, "After");

    cout << "Global memory kernel time: " << time_global << " ms\n\n";

    // ===============================
    // ТЕСТ 2: РАЗДЕЛЯЕМАЯ ПАМЯТЬ
    // ===============================
    cout << "=== SHARED MEMORY VERSION ===\n";
    print_edges(h_data, "Before");

    // Снова копируем исходные данные на GPU
    CUDA_CHECK(cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice));

    CUDA_CHECK(cudaEventRecord(start)); // Старт таймера
    multiply_shared<<<grid, BLOCK_SIZE>>>(d_data, multiplier, N); // Запуск ядра
    CUDA_CHECK(cudaEventRecord(stop));  // Стоп таймера

    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaEventSynchronize(stop));
    CUDA_CHECK(cudaEventElapsedTime(&time_shared, start, stop));

    CUDA_CHECK(cudaMemcpy(h_res, d_data, size, cudaMemcpyDeviceToHost));
    print_edges(h_res, "After");

    cout << "Shared memory kernel time: " << time_shared << " ms\n\n";

    // ===============================
    // ИТОГОВЫЙ ОТЧЕТ
    // ===============================
    cout << "===== PERFORMANCE SUMMARY =====\n";
    cout << "Array size: " << N << " elements\n";
    cout << "Global memory time: " << time_global << " ms\n";
    cout << "Shared memory time: " << time_shared << " ms\n";
    // Считаем коэффициент ускорения
    cout << "Speedup (global / shared): " << time_global / time_shared << "x\n";

    // Освобождение ресурсов
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_data); // Очистка памяти видеокарты
    free(h_data);     // Очистка оперативной памяти
    free(h_res);

    return 0;
}

Writing assignment3_task1.cu


In [4]:
!nvcc -arch=sm_75 assignment3_task1.cu -o task1
!./task1

=== GLOBAL MEMORY VERSION ===
Before
First 5: 17382 95830 33318 97588 65518 
Last 5:  96501 88879 54064 67620 25568 

After
First 5: 43455 239575 83295 243970 163795 
Last 5:  241252 222198 135160 169050 63920 

Global memory kernel time: 0.105088 ms

=== SHARED MEMORY VERSION ===
Before
First 5: 17382 95830 33318 97588 65518 
Last 5:  96501 88879 54064 67620 25568 

After
First 5: 43455 239575 83295 243970 163795 
Last 5:  241252 222198 135160 169050 63920 

Shared memory kernel time: 0.043584 ms

===== PERFORMANCE SUMMARY =====
Array size: 1000000 elements
Global memory time: 0.105088 ms
Shared memory time: 0.043584 ms
Speedup (global / shared): 2.41116x


# Задание 2

In [5]:
%%writefile assignment3_task2.cu
#include <cuda_runtime.h> // Заголовочный файл для функций управления памятью и устройствами CUDA
#include <iostream>       // Стандартный поток ввода-вывода C++
#include <random>         // Современные средства C++ для генерации случайных чисел
#include <cstdlib>        // Стандартные функции C (malloc, free, exit)

#define N 1000000         // Длина векторов (1 миллион элементов)
#define RAND_MIN_VAL 1    // Нижняя граница случайных чисел
#define RAND_MAX_VAL 100  // Верхняя граница случайных чисел

using namespace std;

// ===============================
// Макрос для проверки ошибок выполнения функций CUDA
// ===============================
#define CUDA_CHECK(call) \
do { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        cerr << "CUDA error: " << cudaGetErrorString(err) \
             << " at " << __FILE__ << ":" << __LINE__ << endl; \
        exit(EXIT_FAILURE); \
    } \
} while (0)

// ===============================
// Ядро (Kernel): сложение векторов
// ===============================
// __global__ указывает, что функция вызывается с CPU и исполняется на GPU
__global__ void vector_add(const float* a,
                           const float* b,
                           float* c,
                           int n) {
    // Вычисляем глобальный индекс потока: индекс блока * размер блока + индекс потока в блоке
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Проверка границ, чтобы потоки не вышли за пределы массива N
    if (idx < n) {
        c[idx] = a[idx] + b[idx]; // Каждый поток складывает один элемент
    }
}

// ===============================
// Вспомогательная функция для вывода первых и последних 5 элементов массива
// ===============================
void print_edges(const float* a, const string& msg) {
    cout << msg << "\nFirst 5: ";
    for (int i = 0; i < 5; i++) cout << a[i] << " "; // Начало массива
    cout << "\nLast 5:  ";
    for (int i = N - 5; i < N; i++) cout << a[i] << " "; // Конец массива
    cout << "\n\n";
}

// ===============================
// Функция запуска теста для конкретного размера блока
// ===============================
float run_test(int blockSize,
               const float* h_a,
               const float* h_b,
               float* h_c,
               float* d_a,
               float* d_b,
               float* d_c) {

    // Рассчитываем количество блоков в сетке (Grid), чтобы покрыть N элементов
    int gridSize = (N + blockSize - 1) / blockSize;

    // Создаем события CUDA для точного измерения времени работы GPU
    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));

    // Записываем событие начала
    CUDA_CHECK(cudaEventRecord(start));

    // Запуск ядра с заданными параметрами сетки и блока
    vector_add<<<gridSize, blockSize>>>(d_a, d_b, d_c, N);

    // Записываем событие окончания
    CUDA_CHECK(cudaEventRecord(stop));

    // Проверяем наличие ошибок при запуске ядра
    CUDA_CHECK(cudaGetLastError());

    // Ждем, пока GPU закончит выполнение всех задач до события stop
    CUDA_CHECK(cudaEventSynchronize(stop));

    // Вычисляем разницу во времени между событиями
    float time_ms = 0.0f;
    CUDA_CHECK(cudaEventElapsedTime(&time_ms, start, stop));

    // Копируем результат сложения с GPU обратно на CPU для возможной проверки
    CUDA_CHECK(cudaMemcpy(h_c, d_c, N * sizeof(float), cudaMemcpyDeviceToHost));

    // Удаляем события для освобождения ресурсов
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return time_ms; // Возвращаем время выполнения в миллисекундах
}

// ===============================
// Главная функция программы
// ===============================
int main() {
    // Проверяем наличие GPU с поддержкой CUDA
    int deviceCount = 0;
    CUDA_CHECK(cudaGetDeviceCount(&deviceCount));
    if (deviceCount == 0) {
        cerr << "ERROR: No CUDA GPU found.\n";
        return 1;
    }

    size_t size = N * sizeof(float); // Объем памяти для массивов

    // Указатели для памяти на хосте (CPU) и устройстве (GPU)
    float *h_a, *h_b, *h_c;
    float *d_a, *d_b, *d_c;

    // Инициализация генератора случайных чисел
    random_device rd;
    mt19937 gen(rd());
    uniform_int_distribution<> dist(RAND_MIN_VAL, RAND_MAX_VAL);

    // Выделение оперативной памяти на CPU
    h_a = (float*)malloc(size);
    h_b = (float*)malloc(size);
    h_c = (float*)malloc(size);

    // Заполнение входных массивов случайными числами
    for (int i = 0; i < N; i++) {
        h_a[i] = static_cast<float>(dist(gen));
        h_b[i] = static_cast<float>(dist(gen));
    }

    // Выделение видеопамяти на GPU
    CUDA_CHECK(cudaMalloc(&d_a, size));
    CUDA_CHECK(cudaMalloc(&d_b, size));
    CUDA_CHECK(cudaMalloc(&d_c, size));

    // Копирование исходных данных из RAM (CPU) в VRAM (GPU)
    CUDA_CHECK(cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice));

    // Вывод исходных значений (краев)
    print_edges(h_a, "Array A");
    print_edges(h_b, "Array B");

    // Список размеров блоков, которые мы хотим протестировать
    int blockSizes[] = {32, 64, 128, 256, 512};

    cout << "===== PERFORMANCE RESULTS =====\n";
    cout << "Array size: " << N << " elements\n\n";

    // Цикл по разным размерам блока для сравнения производительности
    for (int bs : blockSizes) {
        float time = run_test(bs, h_a, h_b, h_c, d_a, d_b, d_c);
        cout << "Block size " << bs
             << ": " << time << " ms\n";
    }

    // Вывод итогового массива C (краев) для подтверждения корректности
    print_edges(h_c, "\nResult array C = A + B");

    // Освобождение памяти на видеокарте
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    // Освобождение памяти в системе
    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}

Writing assignment3_task2.cu


In [6]:
!nvcc -arch=sm_75 assignment3_task2.cu -o task2
!./task2

Array A
First 5: 56 88 92 68 55 
Last 5:  50 80 38 9 90 

Array B
First 5: 5 20 33 1 87 
Last 5:  40 73 69 80 58 

===== PERFORMANCE RESULTS =====
Array size: 1000000 elements

Block size 32: 0.199392 ms
Block size 64: 0.058624 ms
Block size 128: 0.054048 ms
Block size 256: 0.055968 ms
Block size 512: 0.056896 ms

Result array C = A + B
First 5: 61 108 125 69 142 
Last 5:  90 153 107 89 148 



# Задание 3

In [7]:
%%writefile assignment3_task3.cu
#include <cuda_runtime.h> // Библиотека среды выполнения CUDA
#include <iostream>       // Стандартный ввод-вывод
#include <random>         // Генерация случайных чисел

// Константы
#define N 1000000         // Количество элементов в массиве
#define BLOCK_SIZE 256    // Количество потоков в одном блоке
#define RAND_MIN_VAL 1    // Минимальное случайное значение
#define RAND_MAX_VAL 100  // Максимальное случайное значение

using namespace std;

// -----------------------------
// Макрос для автоматической проверки ошибок CUDA-функций
// -----------------------------
#define CUDA_CHECK(call) \
do { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        cerr << "CUDA error: " << cudaGetErrorString(err) \
             << " at " << __FILE__ << ":" << __LINE__ << endl; \
        exit(EXIT_FAILURE); \
    } \
} while(0)

// -----------------------------
// Ядра (Kernels)
// -----------------------------

// 1. Ядро с коалесцированным доступом (Эффективно)
// Потоки одного варпа (группа из 32 потоков) обращаются к соседним ячейкам памяти.
// Контроллер памяти может объединить эти запросы в одну транзакцию.
__global__ void coalesced_kernel(float* data, float factor, int n) {
    // Стандартное вычисление индекса: поток 0 -> элемент 0, поток 1 -> элемент 1
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if(idx < n) data[idx] *= factor;
}

// 2. Ядро с некоалесцированным доступом (Неэффективно)
// Здесь индекс вычисляется так, что соседние потоки читают данные из очень далеких участков памяти.
// Это заставляет видеокарту выполнять множество отдельных транзакций вместо одной.
__global__ void uncoalesced_kernel(float* data, float factor, int n) {
    // Нарочито "плохая" формула индексации
    int idx = threadIdx.x * gridDim.x + blockIdx.x;
    if(idx < n) data[idx] *= factor;
}

// -----------------------------
// Вспомогательная функция для печати данных
// -----------------------------
void print_edges(const float* a, const string& msg) {
    cout << msg << "\nFirst 5: ";
    for(int i=0;i<5;i++) cout<<a[i]<<" "; // Печать начала
    cout<<"\nLast 5: ";
    for(int i=N-5;i<N;i++) cout<<a[i]<<" "; // Печать конца
    cout<<"\n\n";
}

// -----------------------------
// Основная функция
// -----------------------------
int main() {
    // Проверка наличия видеокарты
    int deviceCount = 0;
    CUDA_CHECK(cudaGetDeviceCount(&deviceCount));
    if(deviceCount==0) { cerr<<"No CUDA GPU found\n"; return 1; }

    float *h_data, *h_res, *d_data; // Указатели для CPU (h_) и GPU (d_)
    size_t size = N*sizeof(float);  // Общий объем памяти в байтах
    float factor = 2.5f;            // Множитель

    // Инициализация случайных чисел на хосте
    random_device rd;
    mt19937 gen(rd());
    uniform_int_distribution<> dist(RAND_MIN_VAL, RAND_MAX_VAL);

    h_data = (float*)malloc(size); // Выделение RAM
    h_res  = (float*)malloc(size); // Память для хранения результата после копирования
    for(int i=0;i<N;i++) h_data[i] = static_cast<float>(dist(gen));

    // Выделение видеопамяти
    CUDA_CHECK(cudaMalloc(&d_data, size));

    // Расчет количества блоков
    int grid = (N + BLOCK_SIZE -1)/BLOCK_SIZE;

    // Создание событий для замера времени
    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));
    float time_coalesced = 0.0f;
    float time_uncoalesced = 0.0f;

    // ========================
    // ТЕСТ 1: КОАЛЕСЦИРОВАННЫЙ ДОСТУП
    // ========================
    cout<<"=== COALESCED ACCESS ===\n";
    print_edges(h_data,"Before");

    // Копирование данных на GPU
    CUDA_CHECK(cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice));

    CUDA_CHECK(cudaEventRecord(start)); // Запуск секундомера
    coalesced_kernel<<<grid,BLOCK_SIZE>>>(d_data,factor,N); // Вызов ядра
    CUDA_CHECK(cudaEventRecord(stop));  // Остановка секундомера

    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaEventSynchronize(stop)); // Синхронизация CPU и GPU
    CUDA_CHECK(cudaEventElapsedTime(&time_coalesced,start,stop)); // Расчет времени в мс

    // Копирование результата обратно для проверки
    CUDA_CHECK(cudaMemcpy(h_res,d_data,size,cudaMemcpyDeviceToHost));
    print_edges(h_res,"After");
    cout<<"Time: "<<time_coalesced<<" ms\n\n";

    // ========================
    // ТЕСТ 2: НЕКОАЛЕСЦИРОВАННЫЙ ДОСТУП
    // ========================
    cout<<"=== UNCOALESCED ACCESS ===\n";
    print_edges(h_data,"Before");

    // Сброс данных на GPU в исходное состояние
    CUDA_CHECK(cudaMemcpy(d_data,h_data,size,cudaMemcpyHostToDevice));

    CUDA_CHECK(cudaEventRecord(start));
    uncoalesced_kernel<<<grid,BLOCK_SIZE>>>(d_data,factor,N); // Вызов "плохого" ядра
    CUDA_CHECK(cudaEventRecord(stop));

    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaEventSynchronize(stop));
    CUDA_CHECK(cudaEventElapsedTime(&time_uncoalesced,start,stop));

    CUDA_CHECK(cudaMemcpy(h_res,d_data,size,cudaMemcpyDeviceToHost));
    print_edges(h_res,"After");
    cout<<"Time: "<<time_uncoalesced<<" ms\n\n";

    // ========================
    // РАСЧЕТ ЗАМЕДЛЕНИЯ
    // ========================
    float slowdown = time_uncoalesced / time_coalesced;
    cout<<"===== PERFORMANCE COMPARISON =====\n";
    cout<<"Coalesced access time:   "<<time_coalesced<<" ms\n";
    cout<<"Uncoalesced access time: "<<time_uncoalesced<<" ms\n";
    cout<<"Slowdown (uncoalesced / coalesced): "<<slowdown<<"x\n\n";

    // Очистка памяти и уничтожение событий
    CUDA_CHECK(cudaEventDestroy(start));
    CUDA_CHECK(cudaEventDestroy(stop));
    cudaFree(d_data);
    free(h_data);
    free(h_res);

    return 0;
}

Writing assignment3_task3.cu


In [8]:
!nvcc -arch=sm_75 assignment3_task3.cu -o task3
!./task3

=== COALESCED ACCESS ===
Before
First 5: 35 68 73 84 92 
Last 5: 94 22 73 21 10 

After
First 5: 87.5 170 182.5 210 230 
Last 5: 235 55 182.5 52.5 25 

Time: 0.077408 ms

=== UNCOALESCED ACCESS ===
Before
First 5: 35 68 73 84 92 
Last 5: 94 22 73 21 10 

After
First 5: 87.5 170 182.5 210 230 
Last 5: 235 55 182.5 52.5 25 

Time: 0.274912 ms

===== PERFORMANCE COMPARISON =====
Coalesced access time:   0.077408 ms
Uncoalesced access time: 0.274912 ms
Slowdown (uncoalesced / coalesced): 3.55147x



# Задание 4

In [9]:
%%writefile assignment3_task4.cu
#include <cuda_runtime.h> // Библиотека среды выполнения CUDA для работы с GPU
#include <iostream>       // Стандартная библиотека ввода-вывода
#include <random>         // Библиотека для генерации случайных чисел
#include <vector>         // Контейнер вектор для хранения результатов
#include <iomanip>        // Библиотека для форматирования вывода (например, точность чисел)

#define N 1000000         // Общее количество элементов в массивах
#define RAND_MIN_VAL 1    // Минимальное значение случайного числа
#define RAND_MAX_VAL 10000 // Максимальное значение случайного числа

using namespace std;

// -----------------------------
// Макрос для проверки ошибок выполнения CUDA
// -----------------------------
#define CUDA_CHECK(call) \
do { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        cerr << "CUDA error: " << cudaGetErrorString(err) \
             << " at " << __FILE__ << ":" << __LINE__ << endl; \
        exit(EXIT_FAILURE); \
    } \
} while(0)

// -----------------------------
// Ядро (Kernel): параллельное сложение векторов
// -----------------------------
__global__ void vector_add(const float* a, const float* b, float* c, int n) {
    // Рассчитываем уникальный глобальный индекс потока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    // Проверка, чтобы индекс не вышел за пределы массива
    if(idx < n) c[idx] = a[idx] + b[idx];
}

// -----------------------------
// Вспомогательная функция для вывода краев массива (начала и конца)
// -----------------------------
void print_edges(const float* a, const string& msg) {
    cout << msg << "\nFirst 5: ";
    for(int i=0;i<5;i++) cout<<a[i]<<" "; // Печать первых 5 элементов
    cout<<"\nLast 5: ";
    for(int i=N-5;i<N;i++) cout<<a[i]<<" "; // Печать последних 5 элементов
    cout<<"\n\n";
}

// -----------------------------
// Функция запуска теста для конкретного размера блока
// -----------------------------
float run_test(int blockSize, const float* h_a, const float* h_b, float* h_c,
               float* d_a, float* d_b, float* d_c) {
    // Вычисляем размер сетки (Grid) в зависимости от размера блока
    int gridSize = (N + blockSize - 1)/blockSize;

    // Переменные для событий (таймеров)
    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));

    // Записываем начало события
    CUDA_CHECK(cudaEventRecord(start));
    // Запуск ядра на GPU
    vector_add<<<gridSize, blockSize>>>(d_a, d_b, d_c, N);
    // Записываем окончание события
    CUDA_CHECK(cudaEventRecord(stop));

    // Проверка на наличие ошибок запуска
    CUDA_CHECK(cudaGetLastError());
    // Ожидание завершения работы GPU
    CUDA_CHECK(cudaEventSynchronize(stop));

    float time_ms;
    // Получаем время в миллисекундах между событиями
    CUDA_CHECK(cudaEventElapsedTime(&time_ms, start, stop));

    // Копируем полученный результат из GPU в CPU (хост)
    CUDA_CHECK(cudaMemcpy(h_c, d_c, N*sizeof(float), cudaMemcpyDeviceToHost));

    // Освобождаем ресурсы событий
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return time_ms; // Возвращаем время выполнения
}

// -----------------------------
// Основная логика программы
// -----------------------------
int main() {
    // Проверка наличия устройств CUDA
    int deviceCount = 0;
    CUDA_CHECK(cudaGetDeviceCount(&deviceCount));
    if(deviceCount==0) { cerr<<"No CUDA GPU found\n"; return 1; }

    size_t size = N * sizeof(float); // Размер памяти для массива в байтах
    float *h_a, *h_b, *h_c;          // Указатели для оперативной памяти (Host)
    float *d_a, *d_b, *d_c;          // Указатели для видеопамяти (Device)

    // Инициализация генератора случайных чисел
    random_device rd; mt19937 gen(rd()); uniform_int_distribution<> dist(RAND_MIN_VAL, RAND_MAX_VAL);

    // Выделение памяти на CPU
    h_a = (float*)malloc(size);
    h_b = (float*)malloc(size);
    h_c = (float*)malloc(size);

    // Заполнение входных массивов случайными числами
    for(int i=0;i<N;i++) { h_a[i] = dist(gen); h_b[i] = dist(gen); }

    // Выделение памяти на GPU
    CUDA_CHECK(cudaMalloc(&d_a,size));
    CUDA_CHECK(cudaMalloc(&d_b,size));
    CUDA_CHECK(cudaMalloc(&d_c,size));

    // Копирование входных векторов из CPU в GPU
    CUDA_CHECK(cudaMemcpy(d_a,h_a,size,cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_b,h_b,size,cudaMemcpyHostToDevice));

    // Вывод исходных данных
    print_edges(h_a,"Array A");
    print_edges(h_b,"Array B");

    // ===============================
    // Тестирование различных размеров блоков
    // ===============================
    vector<int> blockSizes = {32, 64, 128, 256, 512, 1024}; // Список размеров для проверки
    vector<float> times(blockSizes.size());                 // Массив для хранения времени
    float minTime = 1e9;                                    // Начальное значение для поиска минимума
    int optimalBlock = 0;                                   // Переменная для хранения лучшего размера блока

    cout<<"===== PERFORMANCE RESULTS =====\n";
    for(size_t i=0;i<blockSizes.size();i++) {
        // Выполняем тест и сохраняем время
        times[i] = run_test(blockSizes[i],h_a,h_b,h_c,d_a,d_b,d_c);
        cout<<"Block size "<<blockSizes[i]<<": "<<times[i]<<" ms\n";

        // Поиск минимального времени
        if(times[i] < minTime) {
            minTime = times[i];
            optimalBlock = blockSizes[i];
        }
    }

    // Вывод контрольного результата вычислений
    print_edges(h_c,"\nResult array C = A + B");

    // ===============================
    // Расчет эффективности
    // ===============================
    cout<<"\n===== OPTIMAL CONFIGURATION =====\n";
    cout<<"Optimal block size: "<<optimalBlock<<"\n";
    cout<<"Minimum execution time: "<<minTime<<" ms\n";

    cout<<"\n===== SPEEDUP RELATIVE TO OPTIMAL BLOCK =====\n";
    cout << fixed << setprecision(2); // Установка точности до 2 знаков после запятой
    for(size_t i=0;i<blockSizes.size();i++) {
        // Расчет замедления/ускорения (насколько текущее время больше оптимального)
        float speedup = times[i]/minTime;
        cout<<"Block size "<<blockSizes[i]<<": "<<speedup<<"x\n";
    }

    // Очистка памяти
    CUDA_CHECK(cudaFree(d_a));
    CUDA_CHECK(cudaFree(d_b));
    CUDA_CHECK(cudaFree(d_c));
    free(h_a); free(h_b); free(h_c);

    return 0;
}

Writing assignment3_task4.cu


In [10]:
!nvcc -arch=sm_75 assignment3_task4.cu -o task4
!./task4

Array A
First 5: 9649 2541 1538 5138 2194 
Last 5: 2366 9110 7290 5002 3505 

Array B
First 5: 7536 88 9876 8274 526 
Last 5: 9326 4722 3394 6788 1021 

===== PERFORMANCE RESULTS =====
Block size 32: 0.20432 ms
Block size 64: 0.062464 ms
Block size 128: 0.055296 ms
Block size 256: 0.054144 ms
Block size 512: 0.056672 ms
Block size 1024: 0.059296 ms

Result array C = A + B
First 5: 17185 2629 11414 13412 2720 
Last 5: 11692 13832 10684 11790 4526 


===== OPTIMAL CONFIGURATION =====
Optimal block size: 256
Minimum execution time: 0.054144 ms

===== SPEEDUP RELATIVE TO OPTIMAL BLOCK =====
Block size 32: 3.77x
Block size 64: 1.15x
Block size 128: 1.02x
Block size 256: 1.00x
Block size 512: 1.05x
Block size 1024: 1.10x
